In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 4.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619450 sha256=7e5c49dc61d05b2da0990c13926a00d6c46a84bea5eb60aa64a238f058355e7f
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
pip install pickle-mixin

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6008 sha256=50ba05d25bafff2d7f9617be4707fde8815935ceb365af9eee9dbca0df89673b
  Stored in directory: /root/.cache/pip/wheels/d0/70/0b/673e09a7ed429660d22352a1b117b4f616a8fc054bdd7eb157
Successfully built pickle-mixin


In [ ]:
import os
import sys
import pickle

filenamelst = ['a.txt','b.py','c.ipynb']
filenamelst_abspathname = os.path.abspath('filenamelst.pickle')
print(filenamelst_abspathname)
picklefile = open(str(filenamelst_abspathname),'wb')
pickle.dump(filenamelst, picklefile)
picklefile.close()

/content/filenamelst.pickle


In [ ]:
from flask import Flask, request, render_template
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.snowball import SnowballStemmer
import pandas as pd
import numpy as np
import pickle
import sys
from surprise import Dataset
from surprise import Reader
from surprise import SVD

#pickle part 


def stem_and_vectorize_products_based_on_metadata(product_input):

    word_list = nltk.word_tokenize(product_input)
    input_stemmed = ' '.join([stemmer.stem(word) for word in word_list])
    vec = new_stem_count_vec.transform(np.array(input_stemmed).reshape(1,))
    
    simil = cosine_similarity(vec, new_stem_count_vec_matrix)
    simil_shape = simil.reshape(new_stem_count_vec_matrix.shape[0],)
    simil_scores = pd.DataFrame(data=simil_shape, index=prod_index, columns=['score'])

    # Don't return scores of zero, only as many positive scores as exist
    non_zero_scores = simil_scores[simil_scores['score'] > 0]
    
    if len(non_zero_scores) == 0:
        return 0, 'None'
    
    if len(non_zero_scores) < 10:
        item_count = len(non_zero_scores)
    else:
        item_count = 10
    
    similarity_scores = simil_scores.sort_values(['score'], ascending=False)[:item_count]

    return item_count, (products_desc.iloc[list(similarity_scores.index)]).to_html(index=False, justify='center', classes='table1', border=2)

def get_sample_product(aisle=None):
    if aisle:
        product = products_desc[products_desc['Aisle'].str.contains(aisle)].sample(1)
    else:
        product = products_desc.sample(1)
    name = product['Product Name'].iloc[0]
    prod_aisle = product['Aisle'].iloc[0]
    prod_id = product['Product ID'].iloc[0]  
    return name, prod_aisle, prod_id

def generate_recs(ratings_list, n_to_rec, percent_diverse, rec_aisle=None):
    # Convert ratings list to user_ratings
    userID = 300000
    user_rating =[]
    for product, rating in ratings_list:
        rating_one_product = {'user_id':userID,'product_id':product,'rating':rating}
        user_rating.append(rating_one_product) 

    # add the new ratings to the original ratings DataFrame
    print('Creating ratings dataset...')
    new_ratings_df = new_rec_df.append(user_rating, ignore_index=True)
    new_data = Dataset.load_from_df(new_ratings_df, reader)
    
    # train a model using the new combined DataFrame
    print('Training recommendation model...')
    new_user_svd = SVD(n_factors = 20, n_epochs = 10, lr_all = 0.005, reg_all = 0.4)
    new_user_svd.fit(new_data.build_full_trainset())
    
    # make predictions for the user
    print('Making predictions...')
    list_of_products = []
    for product in new_ratings_df['product_id'].unique():
        product_name = products_desc[products_desc['Product ID'] == product]['Product Name'].iloc[0]
        product_aisle = products_desc[products_desc['Product ID'] == product]['Aisle'].iloc[0]
        list_of_products.append((product, round(new_user_svd.predict(300000, product)[3], 3), product_name, product_aisle))
    
    # order the predictions from highest to lowest rated
    ranked_products = sorted(list_of_products, key=lambda x:x[1], reverse=True)
    
    # return the top n recommendation
    num_results, svd_recs = recommend_diverse_products(ranked_products, n_to_rec, aisle=rec_aisle, percent_diverse=percent_diverse)
    print('Complete')
    return num_results, svd_recs

def grocery_rater(df, num, aisle=None):
    userID = 300000
    rating_list = []
    while num > 0:
        if aisle:
            product = df[df['Aisle'].str.contains(aisle)].sample(1)
        else:
            product = df.sample(1)
        print('\n', product['Product Name'].iloc[0])
        rating = input('How do you rate this product on a scale of 1-5, choose 0 to rate a different product:\n')
        if rating == '0':
            continue
        else:
            rating_one_product = {'user_id':userID,'product_id':product['Product ID'].iloc[0],'rating':int(rating)}
            rating_list.append(rating_one_product) 
            num -= 1
    return rating_list

# return the top n diverse recommendations 
def recommend_diverse_products(ranked_products, n, aisle=None, percent_diverse=.20):
    
    num_diverse = round(n * percent_diverse)
    recs = []
    
    if n < 1:
        print('Number of recommended products must be 1 or more')
        return 0, "None"
    
    for idx, rec in enumerate(ranked_products):
        
        if n == 0:
            recommendation = pd.DataFrame(recs, columns=['Product ID', 'Rating', 'Product Name', 'Aisle'])
            return len(recs), recommendation.to_html(index=False, justify='center', classes='table1', border=2)
            
        prod_id, _, prod_name, aisle_name = [*rec]
        
               
        if aisle:                                    # Did we specify an aisle? 
            if aisle in aisle_name:                  # Is it in the aisle we want?
                if n > num_diverse:                  # Are we looking for a long tail product? No
                    name = prod_name
                    print('Recommendation # ', idx+1, ': ', name, '\n')
                    recs.append(rec)
                    n-= 1
                else:                                 # Are we looking for a long tail product? Yes
                    if prod_id not in short_head:     # Is it NOT in the short_head list?
                        name = prod_name
                        print('Recommendation # ', idx+1, ': ', name, '\n')
                        recs.append(rec)
                        n-= 1
                    else:
                        continue
            elif idx == len(ranked_products)-1:
                print('No recommended products found')
                continue
        else:
            if n > num_diverse:                  # Are we looking for a long tail product? No
                name = prod_name
                print('Recommendation # ', idx+1, ': ', name, '\n')
                recs.append(rec)
                n-= 1
            else:                                 # Are we looking for a long tail product? Yes
                if prod_id not in short_head:     # Is it NOT in the short_head list?
                    name = prod_name
                    print('Recommendation # ', idx+1, ': ', name, '\n')
                    recs.append(rec)
                    n-= 1
                else:
                    continue

def generate_new_user_recommendations(n_to_rate, n_to_rec, percent_diverse, 
                                      rate_aisle=None, rec_aisle=None):
    # Get user ratings
    user_rating = grocery_rater(products_desc, n_to_rate, aisle=rate_aisle)

    # add the new ratings to the original ratings DataFrame
    print('Creating ratings dataset...')
    new_ratings_df = new_rec_df.append(user_rating, ignore_index=True)
    new_data = Dataset.load_from_df(new_ratings_df, reader)
    
    # train a model using the new combined DataFrame
    print('Training recommendation model...')
    new_user_svd = SVD(n_factors = 20, n_epochs = 10, lr_all = 0.005, reg_all = 0.4)
    new_user_svd.fit(new_data.build_full_trainset())
    
    # make predictions for the user
    print('Making predictions...')
    list_of_products = []
    for product in new_ratings_df['product_id'].unique():
        product_name = products_desc[products_desc['Product ID'] == product]['Product Name'].iloc[0]
        product_aisle = products_desc[products_desc['Product ID'] == product]['Aisle'].iloc[0]
        list_of_products.append((product, round(new_user_svd.predict(300000, product)[3], 3), product_name, product_aisle))
    
    # order the predictions from highest to lowest rated
    ranked_products = sorted(list_of_products, key=lambda x:x[1], reverse=True)
    
    # return the top n recommendation
    num_results, svd_recs = recommend_diverse_products(ranked_products, n_to_rec, aisle=rec_aisle, percent_diverse=percent_diverse)
    return num_results, svd_recs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import time
from datetime import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:

# Set default visualization parameters

CB91_Blue = '#2CBDFE'
CB91_Green = '#47DBCD'
CB91_Pink = '#F3A0F2'
CB91_Purple = '#9D2EC5'
CB91_Violet = '#661D98'
CB91_Amber = '#F5B14C'
color_list = [CB91_Blue, CB91_Pink, CB91_Green, CB91_Amber, CB91_Purple, CB91_Violet]
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=color_list)
sns.set_context("notebook", rc={"font.size":16, "axes.titlesize":20, "axes.labelsize":18})
sns.set(font='Franklin Gothic Book',
rc={'axes.axisbelow': False,
'axes.edgecolor': 'lightgrey',
# 'axes.edgecolor': 'white',
'axes.facecolor': 'None',
'axes.grid': False,
'axes.labelcolor': 'dimgrey',
# 'axes.labelcolor': 'white',
'axes.spines.right': False,
'axes.spines.top': False,
'axes.prop_cycle': plt.cycler(color=color_list),
'figure.facecolor': 'white',
'lines.solid_capstyle': 'round',
'patch.edgecolor': 'w',
'patch.force_edgecolor': True,
'text.color': 'dimgrey',
# 'text.color': 'white',    
'xtick.bottom': False,
'xtick.color': 'dimgrey',
# 'xtick.color': 'white',    
'xtick.direction': 'out',
'xtick.top': False,
'ytick.color': 'dimgrey',
# 'ytick.color': 'white',
'ytick.direction': 'out',
'ytick.left': False,
'ytick.right': False})
%matplotlib inline

In [ ]:
aisles = pd.read_csv("/content/aisles.csv")
departments = pd.read_csv("/content/departments.csv")
order_products_prior = pd.read_csv("/content/order_products__prior.csv")
order_products_train = pd.read_csv("/content/order_products__train.csv")
orders = pd.read_csv("/content/orders.csv")
products = pd.read_csv("/content/products.csv")

Merge order_products together
Output: merged_order_products, order_products_desc



In [ ]:
merged_order_products = pd.merge(order_products_train, order_products_prior, how = 'outer')
merged_order_products.head() #merging

,order_id,product_id,add_to_cart_order,reordered
0,1,49302.0,1.0,1.0
1,1,11109.0,2.0,1.0
2,1,10246.0,3.0,0.0
3,1,49683.0,4.0,0.0
4,1,43633.0,5.0,1.0


In [ ]:
products_per_order = merged_order_products.groupby('order_id').count()
products_per_order

,product_id,add_to_cart_order,reordered
order_id,,,
1,8,8,8
2,9,9,9
3,8,8,8
4,13,13,13
5,26,26,26
...,...,...,...
1049709,10,10,10
1049712,11,11,11
1049715,6,6,6


In [ ]:
pro = pd.merge(products,aisles,on='aisle_id')
dep= pd.merge(pro,departments,on='department_id')
dep

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks
...,...,...,...,...,...,...
49683,22827,Organic Black Mission Figs,18,10,bulk dried fruits vegetables,bulk
49684,28655,Crystallized Ginger Chunks,18,10,bulk dried fruits vegetables,bulk
49685,30365,Vegetable Chips,18,10,bulk dried fruits vegetables,bulk
49686,38007,Naturally Sweet Plantain Chips,18,10,bulk dried fruits vegetables,bulk


In [ ]:

# This has an entry for each ordered product.  If we combine it with aisle and department we can see which products
# are ordered the most.
order_products_desc = pd.merge(merged_order_products, dep, on = 'product_id')
order_products_desc.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1,49302.0,1.0,1.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,816049,49302.0,7.0,1.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
2,38841,49302.0,5.0,1.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
3,45900,49302.0,19.0,0.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
4,1,11109.0,2.0,1.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs


Merge user orders

In [ ]:
# Now merge the orders with the ordered prodcuts, combining on order_id
# This is a lot of info. We will trim some out before modeling.
merged_orders = pd.merge(orders, order_products_desc, on = 'order_id')
merged_orders.head() # Because the products showing are from the 1st order for this user, they all have Nan in days_since
# Because pd.merge defaults to an inner join, the test orders are not in this dataframe because they had no products.

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,23391,7,prior,17.0,0.0,10.0,28.0,27344.0,11.0,0.0,Uncured Genoa Salami,96,20,lunch meat,deli
1,23391,7,prior,17.0,0.0,10.0,28.0,11520.0,9.0,0.0,Large Alfresco Eggs,86,16,eggs,dairy eggs
2,23391,7,prior,17.0,0.0,10.0,28.0,8277.0,3.0,1.0,Apple Honeycrisp Organic,24,4,fresh fruits,produce
3,23391,7,prior,17.0,0.0,10.0,28.0,4920.0,6.0,1.0,Seedless Red Grapes,123,4,packaged vegetables fruits,produce
4,23391,7,prior,17.0,0.0,10.0,28.0,4945.0,7.0,1.0,Large Pineapple Chunks,123,4,packaged vegetables fruits,produce


In [ ]:
test = orders[orders['eval_set']=='test']
test.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13.0,5.0,15.0,11.0
44,329954,4,test,6.0,3.0,12.0,30.0
53,1528013,6,test,4.0,3.0,16.0,22.0
96,1376945,11,test,8.0,6.0,11.0,8.0
102,1356845,12,test,6.0,1.0,20.0,30.0


In [ ]:
# Now I can see everything a particular user has ordered, and how often
#merged_orders[merged_orders['user_id']==3]['product_name'].value_counts()

Prepare data for recommendation system

In [ ]:
user_item = merged_orders[['user_id', 'product_id', 'reordered']]

In [ ]:
user_item[user_item.duplicated(subset = ['user_id', 'product_id'], keep = 'first')]

,user_id,product_id,reordered
12,7,17638.0,1.0
13,7,13198.0,1.0
18,7,40852.0,1.0
152,66,8143.0,1.0
219,75,13176.0,1.0
...,...,...,...
62570,14495,32156.0,1.0
62579,14495,26308.0,1.0
62582,14495,24832.0,1.0
62588,14495,21512.0,1.0


In [ ]:
data=[]
data_columns=['user_id','product_id','count']
for user in user_item.user_id.unique():
    counts = user_item[user_item['user_id']==user].product_id.value_counts()
    for i in range(len(counts)):
         data.append([user, counts.index[i], counts.values[i]])

In [ ]:
rec_df = pd.DataFrame(data, columns=data_columns)

# Order of a user

In [ ]:
namee=rec_df[rec_df['user_id']==1] #imppp
mer=pd.merge(namee , dep , on='product_id')
mer.product_name

Series([], Name: product_name, dtype: object)

In [ ]:
rec_df['count'].max()

5

In [ ]:
# In case I want a smaller subset of users I'll build users_150K
# But I wonder if pulling out items only ordered once or users who only ordered once would be better
#grouping from 
users_150K = rec_df[rec_df['user_id'] < 150000]

In [ ]:
user_info = merged_orders[['user_id', 'order_number', 'order_dow', 'order_hour_of_day', 
                           'days_since_prior_order', 'aisle']]

In [ ]:
user_data = pd.get_dummies(user_info, prefix=None, columns=['aisle'])

In [ ]:
# Splitting the data in thirds to use groupby then will rejoin them
user_data1 = user_data[user_data['user_id'] <= 65000]
user_data2 = user_data[(user_data['user_id'] <= 135000) & (user_data['user_id'] > 65000)]
user_data3 = user_data[user_data['user_id'] > 135000]

In [ ]:
grouped_user1 = user_data1.groupby('user_id').sum()
grouped_user2 = user_data2.groupby('user_id').sum()
grouped_user3 = user_data3.groupby('user_id').sum()


In [ ]:
# Go back to user_data and drop the aisle info before I group the other variables in different ways
user_data1_noaisles = user_data1.iloc[:,:5]
user_data2_noaisles = user_data2.iloc[:,:5]
user_data3_noaisles = user_data3.iloc[:,:5]

In [ ]:
group1 = user_data1_noaisles.groupby('user_id').agg({'order_number': 'max', 'order_dow': lambda x:x.value_counts().index[0], 
                                            'order_hour_of_day': 'median', 'days_since_prior_order': 'mean'})
group2 = user_data2_noaisles.groupby('user_id').agg({'order_number': 'max', 'order_dow': lambda x:x.value_counts().index[0], 
                                            'order_hour_of_day': 'median', 'days_since_prior_order': 'mean'})
group3 = user_data3_noaisles.groupby('user_id').agg({'order_number': 'max', 'order_dow': lambda x:x.value_counts().index[0], 
                                            'order_hour_of_day': 'median', 'days_since_prior_order': 'mean'})

In [ ]:
# get number of orders for each user and add to grouped_user dfs
grouped_user1['num_orders'] = group1.order_number
grouped_user2['num_orders'] = group2.order_number
grouped_user3['num_orders'] = group3.order_number

grouped_user1['mean_days_since'] = group1.days_since_prior_order
grouped_user2['mean_days_since'] = group2.days_since_prior_order
grouped_user3['mean_days_since'] = group3.days_since_prior_order

grouped_user1['mode_order_dow'] = group1.order_dow
grouped_user2['mode_order_dow'] = group2.order_dow
grouped_user3['mode_order_dow'] = group3.order_dow

grouped_user1['median_order_hour'] = group1.order_hour_of_day
grouped_user2['median_order_hour'] = group2.order_hour_of_day
grouped_user3['median_order_hour'] = group3.order_hour_of_day

grouped_user1.drop(columns=['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True)
grouped_user2.drop(columns=['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True)
grouped_user3.drop(columns=['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True)



In [ ]:
grouped_users = pd.concat([grouped_user1, grouped_user2, grouped_user3], axis=0)

In [ ]:

grouped_users.head()

,aisle_air fresheners candles,aisle_asian foods,aisle_baby accessories,aisle_baby bath body care,aisle_baby food formula,aisle_bakery desserts,aisle_baking ingredients,aisle_baking supplies decor,aisle_beauty,aisle_beers coolers,aisle_body lotions soap,aisle_bread,aisle_breakfast bakery,aisle_breakfast bars pastries,aisle_bulk dried fruits vegetables,aisle_bulk grains rice dried goods,aisle_buns rolls,aisle_butter,aisle_candy chocolate,aisle_canned fruit applesauce,aisle_canned jarred vegetables,aisle_canned meals beans,aisle_canned meat seafood,aisle_cat food care,aisle_cereal,aisle_chips pretzels,aisle_cleaning products,aisle_cocoa drink mixes,aisle_coffee,aisle_cold flu allergy,aisle_condiments,aisle_cookies cakes,aisle_crackers,aisle_cream,aisle_deodorants,aisle_diapers wipes,aisle_digestion,aisle_dish detergents,aisle_dog food care,aisle_doughs gelatins bake mixes,...,aisle_packaged vegetables fruits,aisle_paper goods,aisle_pasta sauce,aisle_pickled goods olives,aisle_plates bowls cups flatware,aisle_popcorn jerky,aisle_poultry counter,aisle_prepared meals,aisle_prepared soups salads,aisle_preserved dips spreads,aisle_protein meal replacements,aisle_red wines,aisle_refrigerated,aisle_refrigerated pudding desserts,aisle_salad dressing toppings,aisle_seafood counter,aisle_shave needs,aisle_skin care,aisle_soap,aisle_soft drinks,aisle_soup broth bouillon,aisle_soy lactosefree,aisle_specialty cheeses,aisle_specialty wines champagnes,aisle_spices seasonings,aisle_spirits,aisle_spreads,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt,num_orders,mean_days_since,mode_order_dow,median_order_hour
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21.0,18.1,0.0,10.0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,4.0,10.0,1.0,14.0
13,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0,9.0,1.0,12.0
23,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4,2.0,9.0,0.0,10.0
24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19.0,0.0,0.0,16.0


In [ ]:
# This function came from a Medium article by Adam Ross Nelson to rearrange columns in a df
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [ ]:
grouped_users[grouped_users['num_orders']<=5]

,aisle_air fresheners candles,aisle_asian foods,aisle_baby accessories,aisle_baby bath body care,aisle_baby food formula,aisle_bakery desserts,aisle_baking ingredients,aisle_baking supplies decor,aisle_beauty,aisle_beers coolers,aisle_body lotions soap,aisle_bread,aisle_breakfast bakery,aisle_breakfast bars pastries,aisle_bulk dried fruits vegetables,aisle_bulk grains rice dried goods,aisle_buns rolls,aisle_butter,aisle_candy chocolate,aisle_canned fruit applesauce,aisle_canned jarred vegetables,aisle_canned meals beans,aisle_canned meat seafood,aisle_cat food care,aisle_cereal,aisle_chips pretzels,aisle_cleaning products,aisle_cocoa drink mixes,aisle_coffee,aisle_cold flu allergy,aisle_condiments,aisle_cookies cakes,aisle_crackers,aisle_cream,aisle_deodorants,aisle_diapers wipes,aisle_digestion,aisle_dish detergents,aisle_dog food care,aisle_doughs gelatins bake mixes,...,aisle_packaged vegetables fruits,aisle_paper goods,aisle_pasta sauce,aisle_pickled goods olives,aisle_plates bowls cups flatware,aisle_popcorn jerky,aisle_poultry counter,aisle_prepared meals,aisle_prepared soups salads,aisle_preserved dips spreads,aisle_protein meal replacements,aisle_red wines,aisle_refrigerated,aisle_refrigerated pudding desserts,aisle_salad dressing toppings,aisle_seafood counter,aisle_shave needs,aisle_skin care,aisle_soap,aisle_soft drinks,aisle_soup broth bouillon,aisle_soy lactosefree,aisle_specialty cheeses,aisle_specialty wines champagnes,aisle_spices seasonings,aisle_spirits,aisle_spreads,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt,num_orders,mean_days_since,mode_order_dow,median_order_hour
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,4.0,10.0,1.0,14.0
13,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0,9.0,1.0,12.0
23,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4,2.0,9.0,0.0,10.0
42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,7.0,1.0,19.0
46,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,7.0,1.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14441,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0,30.0,0.0,20.0
14453,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.0,5.0,3.0,8.0
14454,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,5.0,24.0,3.0,16.0


In [ ]:
products_with_count = rec_df.groupby('product_id').sum().drop(columns='user_id').sort_values(by='count')
products_with_count[products_with_count['count']<=5]  # 2741 products ordered 5 times or less

,count
product_id,
21040.0,1
24291.0,1
24273.0,1
24261.0,1
45054.0,1
...,...
46984.0,5
47325.0,5
45158.0,5


Modeling a recommendation system

In [ ]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import KNNBaseline
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise.prediction_algorithms import SVD, SVDpp
from surprise.model_selection import GridSearchCV
from surprise import accuracy

In [ ]:
reader = Reader(rating_scale=(1, 100))

# The columns must correspond to user id, item id and ratings (in that order).
rec_data = Dataset.load_from_df(rec_df, reader)

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(rec_data, test_size=.25, random_state=12)

In [ ]:

#sim_cos = {'name':'cosine', 'user_based':False}

#tic = time.time()

#basic = knns.KNNBasic(sim_options=sim_cos)
#basic.fit(trainset)

#fit_time = time.time()-tic
#print('fit ', fit_time)

In [ ]:
tic = time.time()

svd_algo = SVD()

svd_algo.fit(trainset)

fit_time = time.time()-tic
print('fit ', fit_time)
toc = time.time()

svd_algo_predictions = svd_algo.test(testset)

pred_time = time.time()-toc
print('pred ', pred_time)

# Then compute RMSE
accuracy.rmse(svd_algo_predictions)

fit  3.2182013988494873
pred  0.28189635276794434
RMSE: 0.2200


0.21997414613782731

In [ ]:

accuracy.rmse(svd_algo_predictions)

RMSE: 0.2200


0.21997414613782731

In [ ]:
rec_df[rec_df['user_id'] == 2555]

,user_id,product_id,count
10600,2555,21903.0,1
10601,2555,45445.0,1
10602,2555,33120.0,1
10603,2555,44632.0,1
10604,2555,3233.0,1
10605,2555,13877.0,1
10606,2555,22400.0,1
10607,2555,21938.0,1
10608,2555,8539.0,1
10609,2555,18531.0,1


In [ ]:
namee=rec_df[rec_df['user_id']==2555] #imppp
mer=pd.merge(namee , dep , on='product_id')
mer.product_name

0                                  Organic Baby Spinach
1                Organic Bagged Mini Dark Peanut Butter
2                                    Organic Egg Whites
3                            Sparkling Water Grapefruit
4     Organic Turkey with Rosemary, Sage & Thyme Bon...
5                                         Smoked Salmon
6                                Peanut Butter, Classic
7                                     Green Bell Pepper
8            Organic Chicken with Lemongrass Bone Broth
9                          Organic Heavy Whipping Cream
10                             Ripe Large Pitted Olives
11                          Total Greek Strained Yogurt
12                  Uncured Hickory Smoked Sunday Bacon
13                              Baby Seedless Cucumbers
14                                 Organic Blackberries
15                                      93% Ground Beef
16                    Classic Almond Butter All Natural
17                                          Half

In [ ]:
uid=196
iid=13176
# actual value was 2, prediction is 1.77
pred = svd_algo.predict(uid, iid, r_ui=2, verbose=True)

user: 196        item: 13176      r_ui = 2.00   est = 1.09   {'was_impossible': False}


In [ ]:
testset[0]

(8241, 10385.0, 1.0)

In [ ]:
pred = svd_algo.predict(127895, 6101, r_ui=2, verbose=True)

user: 127895     item: 6101       r_ui = 2.00   est = 1.05   {'was_impossible': False}


In [ ]:
svd_algo_predictions[0]

Prediction(uid=8241, iid=10385.0, r_ui=1.0, est=1.069266604656487, details={'was_impossible': False})

In [ ]:
param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs_model = GridSearchCV(SVD, param_grid=param_grid, n_jobs = -1, joblib_verbose=5)
gs_model.fit(rec_data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   23.7s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  2.6min finished


In [ ]:
gs_model.best_params

{'mae': {'lr_all': 0.005, 'n_epochs': 10, 'n_factors': 20, 'reg_all': 0.6},
 'rmse': {'lr_all': 0.005, 'n_epochs': 10, 'n_factors': 20, 'reg_all': 0.4}}

In [ ]:
results_df = pd.DataFrame.from_dict(gs_model.cv_results)

In [ ]:
tic = time.time()

svd_best_gs = SVD(n_factors = 100, n_epochs = 10, lr_all = 0.005, reg_all = 0.4)

svd_best_gs.fit(trainset)

fit_time = time.time()-tic
print('fit ', fit_time)
toc = time.time()

svd_best_gs_predictions = svd_best_gs.test(testset)

pred_time = time.time()-toc
print('pred ', pred_time)

# Then compute RMSE
accuracy.rmse(svd_best_gs_predictions)

# RMSE went down on grid search model from 3.4616 to 3.2483

fit  1.489058494567871
pred  0.13265681266784668
RMSE: 0.2153


0.21531261809035102

In [ ]:
max(testset,key=lambda item:item[2])
pred = svd_best_gs.predict(41356,38652, r_ui=100.0, verbose=True)
top_est = 0
top_pred = [0,0,0,0]
for uid, iid, true_r, est, _ in svd_best_gs_predictions:
    if est > top_est:
        top_est = est
        top_pred = [uid,iid,true_r,est]
top_pred

user: 41356      item: 38652      r_ui = 100.00   est = 1.05   {'was_impossible': False}


[6409, 9637.0, 4.0, 1.8942147967716476]

In [ ]:
rec_df['count'].value_counts()

1    57330
2     2297
3      178
4       37
5        4
Name: count, dtype: int64

In [ ]:
rec_df['rating'] = rec_df['count'].apply(lambda x: 5 if (x >= 5) else x)

In [ ]:
new_rec_df = rec_df.drop(columns='count')
new_rec_df.tail()

,user_id,product_id,rating
59841,14502,31717.0,1
59842,14502,37524.0,1
59843,14502,4781.0,1
59844,14502,3142.0,1
59845,14502,13176.0,1


In [ ]:
rec_columns=new_rec_df.columns.values
rec_index=new_rec_df.index.values
rec_user=new_rec_df.user_id.values
rec_rating=new_rec_df.rating.values
rec_prod_id=new_rec_df.product_id.values

In [ ]:
new_df = pd.DataFrame(np.column_stack([rec_user, rec_prod_id, rec_rating]), index=rec_index, columns=rec_columns)
new_df

,user_id,product_id,rating
0,7.0,13198.0,2.0
1,7.0,40852.0,2.0
2,7.0,17638.0,2.0
3,7.0,12053.0,1.0
4,7.0,37602.0,1.0
...,...,...,...
59841,14502.0,31717.0,1.0
59842,14502.0,37524.0,1.0
59843,14502.0,4781.0,1.0
59844,14502.0,3142.0,1.0


In [ ]:
# Must start with a new reader

tic = time.time()
reader = Reader(rating_scale=(1, 5))
readtime = time.time()-tic
print(readtime)
# The columns must correspond to user id, item id and ratings (in that order).
toc = time.time()
new_rec_data = Dataset.load_from_df(new_rec_df, reader)
loadtime = time.time()-toc
print(loadtime)
# sample random trainset and testset
# test set is made of 25% of the ratings.
tac = time.time()
new_trainset, new_testset = train_test_split(new_rec_data, test_size=.25, random_state=12)
splittime = time.time()-tac
print(splittime)

7.009506225585938e-05
0.07197380065917969
0.10776066780090332


In [ ]:
tic = time.time()

new_svd_algo = SVD(n_factors = 100, n_epochs = 10, lr_all = 0.005, reg_all = 0.4)

new_svd_algo.fit(new_trainset)

fit_time = time.time()-tic
print('fit ', fit_time)
toc = time.time()

new_predictions = new_svd_algo.test(new_testset)

pred_time = time.time()-toc
print('pred ', pred_time)

# Then compute RMSE
accuracy.rmse(new_predictions)

# Not bad, RMSE went to to 1.2649'''

fit  1.6332526206970215
pred  0.3791368007659912
RMSE: 0.2155


0.21554694616703457

In [ ]:

accuracy.rmse(new_predictions)

RMSE: 0.2155


0.21554694616703457

In [ ]:
# This prediction was 1.76 before, it has gone up to 2.12 with the new scale

uid=196
iid=13176
# get a prediction for specific users and items.
pred = new_svd_algo.predict(uid, iid, r_ui=2, verbose=True)

user: 196        item: 13176      r_ui = 2.00   est = 1.06   {'was_impossible': False}


In [ ]:
pred = new_svd_algo.predict(41356,38652, r_ui=5.0, verbose=True)

user: 41356      item: 38652      r_ui = 5.00   est = 1.05   {'was_impossible': False}


In [ ]:
# Well lets try another grid search

# Started at 9:57am, ended at 12:30pm

param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
new_gs_model = GridSearchCV(SVD, param_grid=param_grid, n_jobs = -1, joblib_verbose=5)
new_gs_model.fit(new_rec_data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  2.6min finished


In [ ]:
new_gs_model.best_params

{'mae': {'lr_all': 0.005, 'n_epochs': 10, 'n_factors': 20, 'reg_all': 0.6},
 'rmse': {'lr_all': 0.005, 'n_epochs': 10, 'n_factors': 20, 'reg_all': 0.4}}

In [ ]:
new_results_df = pd.DataFrame.from_dict(new_gs_model.cv_results)

In [ ]:
# Creating the model from the best params

tic = time.time()

new_svd_best_gs = SVD(n_factors = 20, n_epochs = 10, lr_all = 0.005, reg_all = 0.4)

new_svd_best_gs.fit(new_trainset)

fit_time = time.time()-tic
print('fit ', fit_time)
toc = time.time()

new_svd_best_gs_predictions = new_svd_best_gs.test(new_testset)

pred_time = time.time()-toc
print('pred ', pred_time)

# Then compute RMSE
accuracy.rmse(new_svd_best_gs_predictions)

# RMSE went down on grid search model from 1.2649 to 1.2648.  lol, not much.

fit  0.7443935871124268
pred  0.13977479934692383
RMSE: 0.2116


0.21158625789927635

In [ ]:
# Get top predictions for a user

from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
# data = Dataset.load_builtin('ml-100k')
# trainset = data.build_full_trainset()
# algo = SVD()
# algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
# anti_testset = new_trainset.build_anti_testset()
# predictions = anti_new_svd_best_gs.test(anti_testset)

In [ ]:
top_n = get_top_n(new_svd_best_gs_predictions, n=10)

In [ ]:
top_n[8241]

[(47209.0, 1.0774504501684734),
 (13176.0, 1.0611994713756505),
 (10385.0, 1.0601021705439067),
 (1039.0, 1)]

**VERY IMPORTANT FUNCTION**

In [ ]:
def prediction(person):
  val=top_n[person]
  val1=[]
  for i in range (len(val)):
    val1.append(val[i][0])
  user=dep.loc[dep['product_id'].isin(val1)]
  pn=user['product_name']
  lst= list(pn)
  return lst

In [ ]:
pickle.dump(top_n, open("Pickle/topn.p", "wb"))

In [ ]:
pickle.dump(dep, open("Pickle/dep.p", "wb"))

In [ ]:
pickle.dump(rec_df, open("Pickle/recdf.p", "wb"))

In [ ]:
prediction(14293)

['Honeygrams, gluten-Free, Wheat-Free Graham Style Crackers',
 'Fair Trade Dark Chocolate Cookies',
 'Organic Lowatf Plain Yogurt',
 'Organic Red Onion',
 'Organic Bosc Pear']

In [ ]:
def most_ordered(person):
  namee=rec_df[rec_df['user_id']==person] #imppp
  mer=pd.merge(namee , dep , on='product_id')
  return list(mer.product_name)

In [ ]:
most_ordered(14293)

['Organic Red Chard Greens',
 'Organic Tomato Cluster',
 'Honeygrams, gluten-Free, Wheat-Free Graham Style Crackers',
 'Limes',
 'Carrots',
 'Fair Trade Dark Chocolate Cookies',
 'Organic Cilantro',
 'Organic Lowatf Plain Yogurt',
 'Organic Bosc Pear',
 'Organic Soft Wheat Bread',
 'Organic Red Onion',
 'Organic Hass Avocado']

In [ ]:
print(orders.loc[orders['user_id']==2555])

       order_id  user_id  ... order_hour_of_day  days_since_prior_order
41495   1435633     2555  ...              13.0                     NaN
41496    610004     2555  ...              17.0                     8.0
41497    241397     2555  ...               7.0                     9.0
41498   1688299     2555  ...              11.0                     3.0
41499   2781552     2555  ...              11.0                     3.0
41500   2019457     2555  ...               8.0                     6.0
41501    590946     2555  ...              11.0                     5.0
41502   1045122     2555  ...              20.0                     5.0
41503   2383389     2555  ...              15.0                     6.0
41504   2465391     2555  ...              17.0                     5.0
41505    561564     2555  ...              10.0                    10.0
41506   3234485     2555  ...              18.0                     2.0
41507   1544656     2555  ...              17.0                 

In [ ]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1.0,2.0,8.0,NaN
1,2398795,1,prior,2.0,3.0,7.0,15.0
2,473747,1,prior,3.0,3.0,12.0,21.0
3,2254736,1,prior,4.0,4.0,7.0,29.0
4,431534,1,prior,5.0,4.0,15.0,28.0
...,...,...,...,...,...,...,...
239661,1994634,14504,prior,7.0,6.0,13.0,6.0
239662,2487022,14504,prior,8.0,0.0,10.0,30.0
239663,1448635,14504,prior,9.0,5.0,18.0,12.0
239664,775696,14504,prior,10.0,1.0,16.0,30.0


In [ ]:
len(top_n.values())

4250

In [ ]:

order_products_desc['product_name'].value_counts()

Banana                                  12608
Bag of Organic Bananas                  10224
Organic Strawberries                     7038
Organic Baby Spinach                     6554
Organic Hass Avocado                     5361
                                        ...  
Bath & Kitchen Multi-Surface Cleaner        1
Spicy Nori Snack                            1
Grow & Gain Banana Shake                    1
Cannoli                                     1
Extra Aged Gouda Cheese                     1
Name: product_name, Length: 34904, dtype: int64

In [ ]:
all_rec_items=[]  # Making a list of all recommended items in the top for each user, then count them
for idx in range(len(top_n.values())-200000):
    all_rec_items.extend([x[0] for x in [*top_n.values()][idx]]) # product_ids recommended for first user
# all_rec_items

In [ ]:
from collections import Counter

ctr = Counter(all_rec_items)
most_ctr = ctr.most_common(20)

In [ ]:
most_ctr
[products_desc[products_desc['product_id']==x]['product_name'].iloc[0] for x in [x[0] for x in most_ctr]]

[]

In [ ]:
new_rec_df.product_id.value_counts().sum() # 13,863,746 ratings, so the top 80% would be 11,091,000

59846

In [ ]:
new_rec_df.product_id.value_counts()[:1].cumsum().values # I admit this was trial and error

array([  787,  1416,  1874, ..., 39590, 39596, 39602])

In [ ]:
# so the top 6200 products account for 80% of the ratings (or purchases in this case)
short_head = list(new_rec_df.product_id.value_counts()[:1].cumsum().index)

In [ ]:

short_head

[24852.0,
 13176.0,
 21137.0,
 21903.0,
 47209.0,
 47766.0,
 47626.0,
 16797.0,
 27966.0,
 26209.0,
 24964.0,
 22935.0,
 39275.0,
 27845.0,
 45007.0,
 30391.0,
 5876.0,
 4920.0,
 4605.0,
 45066.0,
 49683.0,
 40706.0,
 42265.0,
 44632.0,
 28204.0,
 21616.0,
 8518.0,
 31717.0,
 17794.0,
 37646.0,
 21938.0,
 46979.0,
 19057.0,
 8277.0,
 30489.0,
 10749.0,
 48679.0,
 28985.0,
 43961.0,
 41950.0,
 49235.0,
 27104.0,
 44359.0,
 26604.0,
 9076.0,
 22035.0,
 25890.0,
 24184.0,
 43352.0,
 31506.0,
 5450.0,
 27086.0,
 39877.0,
 5077.0,
 8424.0,
 27521.0,
 8174.0,
 19660.0,
 34126.0,
 35951.0,
 34969.0,
 22825.0,
 12341.0,
 46667.0,
 28842.0,
 35221.0,
 24838.0,
 27344.0,
 5785.0,
 15290.0,
 29487.0,
 44142.0,
 39928.0,
 33000.0,
 9839.0,
 33731.0,
 47144.0,
 16759.0,
 20114.0,
 28199.0,
 27156.0,
 432.0,
 3957.0,
 34243.0,
 4799.0,
 21709.0,
 26369.0,
 18465.0,
 34358.0,
 11520.0,
 20995.0,
 33198.0,
 23909.0,
 32689.0,
 42828.0,
 37067.0,
 33754.0,
 37687.0,
 43122.0,
 5479.0,
 41844.0,
 19678.